In [32]:
import pandas as pd

In [44]:
chennai_df = pd.read_csv('Chennai.csv')

hyderabad_df = pd.read_csv('Hyderabad.csv')

kolkata = pd.read_csv('Kolkata.csv')

In [45]:
chennai_df

,seller_type,bedroom,layout_type,property_type,locality,price,area,furnish_type,bathroom
0,OWNER,3.0,BHK,Apartment,Ambattur,"17,000",1570.0,Semi-Furnished,2 bathrooms
1,OWNER,2.0,BHK,Apartment,Kelambakkam,"16,000",1402.0,Semi-Furnished,2 bathrooms
2,OWNER,2.0,BHK,Apartment,Adyar,"20,000",800.0,Semi-Furnished,2 bathrooms
3,OWNER,2.0,BHK,Apartment,Sembakkam,"14,000",1180.0,Unfurnished,2 bathrooms
4,OWNER,1.0,BHK,Independent House,Mogappair,"7,000",400.0,Unfurnished,1 bathrooms
...,...,...,...,...,...,...,...,...,...
9278,OWNER,3.0,BHK,Apartment,Thoraipakkam OMR,"50,000",1850.0,Furnished,3 bathrooms
9279,OWNER,2.0,BHK,Independent Floor,Sithalapakkam,"13,000",1000.0,Semi-Furnished,2 bathrooms
9280,OWNER,1.0,BHK,Independent Floor,Royapettah,"10,000",460.0,Unfurnished,1 bathrooms
9281,OWNER,2.0,BHK,Independent Floor,Ramavaram,"8,000",700.0,Unfurnished,1 bathrooms
